In [ ]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, RocCurveDisplay
from sklearn.metrics import make_scorer, roc_auc_score, recall_score, accuracy_score, confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline

# Loading and cleaning Dataset

In [ ]:
database_file  = ""
output_location =  ""

In [ ]:
def load_clean_dataset(loaded_df, na_perc_limit, columns_to_delete=[]):
    print(loaded_df.shape)
    loaded_df = loaded_df.drop(columns=columns_to_delete)
    tot = loaded_df.shape[0]
    print("Cleaning dataset... \n")
    for col in loaded_df.columns:
        na_per = 1-len(loaded_df[col].dropna())/tot
        if na_per > na_perc_limit:
            print(f"Column {col} --> %NaN = {na_per}. Removed")
            loaded_df = loaded_df.drop(columns=col)
    print("\n Dropping rows with NaNs...")
    loaded_df = loaded_df.dropna()
    print(f"\nFinal columns: {loaded_df.columns}")
    print(loaded_df.shape)
    return loaded_df

columns_to_delete = ["Unnamed: 0", "person_id", "fecha_ingreso_urgencias", "shock_septico", "foco", "sintoma_nan", "fecha_nacimiento", "codigo_postal", "center", "dag"]

def combine_columns(df_to_clean, column_list, new_col_name):
    df_to_clean[new_col_name] = df_to_clean[column_list].sum(axis=1).astype(int)
    clean_df = df_to_clean.drop(columns=column_list)
    return clean_df

df_to_clean = pd.read_csv(database_file)
hepatic_cols = [c for c in df_to_clean.columns if "hepatopatia" in c]
tumor_cols = [c for c in df_to_clean.columns if "cancer" in c]
for new_name, col_list in {"enf_hepaticas": hepatic_cols, "tumores": tumor_cols}.items():
    df_to_clean = combine_columns(df_to_clean, col_list, new_name)

processed_df = load_clean_dataset(
    loaded_df = df_to_clean,
    na_perc_limit = 0.1,
    columns_to_delete = columns_to_delete
)

### Splitting data into train/test

In [ ]:
"""X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)"""

### Defining models and hyperparameters for grid search

In [ ]:
log_dict = {
    "logistic_regression": {
        "model": LogisticRegression(),
        "params": {
            "solver": ["lbfgs", "liblinear", "saga"],
            "C": [0.1, 1, 10, 100, 250, 500],
            "penalty": ["l2", "l1", "elasticnet"],
            "max_iter": [100, 1000, 5000]
        }
    }
}
randfor_dict = {
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "criterion": ["gini", "entropy", "log_loss"],
            "n_estimators": [10, 50, 100, 250, 500],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [5, 10, 20, 30, 50]
        }
    }
}
bayes_dict = {
    "bernoulli_bayes": {
        "model": BernoulliNB(),
        "params": {
            "alpha": [0.1, 0.5, 1.0, 1.5, 2.0],
            "fit_prior": [True, False],
            "binarize": [0.0, 0.5, 1.0, 1.5, 2.0]
        }
    }
}
svc_dict = {
    "svc": {
        "model": SVC(),
        "params": {
            "C": [0.1, 1, 10, 100],
            "kernel": ["linear", "rbf", "poly", "sigmoid"],
            "gamma": ["scale", "auto"]
        }
    }
}
"""grid_search_list = (log_dict, randfor_dict, bayes_dict, grad_boost_dict)
all_grid_models = {k:v for model_dict in model_list for k,v in model_dict.items()}"""

# Models that require normalization

In [ ]:
from scipy.stats import uniform, loguniform, randint

log_dict = {
    "logistic_regression": {
        "model": LogisticRegression(),
        "grid_params": {
            "solver": ["lbfgs", "liblinear", "saga"],
            "C": [0.1, 1, 10, 100, 250, 500],
            "penalty": ["l2", "l1", "elasticnet"],
            "max_iter": [100, 1000, 5000]
        },
        "random_params": {
            "solver": ["liblinear", "saga"],
            "C": loguniform(0.1, 500),
            "penalty": ["l2", "l1", "elasticnet"],
            "max_iter": [100, 1000, 5000]
        }
    }
}

svc_dict = {
    "svc": {
        "model": SVC(),
        "grid_params": {
            "C": [0.1, 1, 10, 100],
            "kernel": ["linear", "rbf", "poly", "sigmoid"],
            "gamma": ["scale", "auto"]
        },
        "random_params": {
            "C": np.logspace(-3, 3, 7),
            "kernel": ["linear", "rbf", "poly", "sigmoid"],
            "degree": [2, 3, 4, 5],          # Only used for "poly" kernel
            "gamma": ["scale", "auto", 0.001, 0.01, 0.1, 1, 10], # Only used for "rbf", "poly", and "sigmoid" kernels
            "coef0": [0.0, 0.1, 0.5, 1.0],  # Only used for "poly" and "sigmoid" kernels
            "class_weight": [None, "balanced"],
        }
    }
}

sgdc_dict = {
    "sgdc_classifier": {
        "model": SGDClassifier(),
        "grid_params": {
            "loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron"],
            "alpha": [0.0001, 0.001, 0.01],
            "max_iter": [100, 1000, 5000],
            "early_stopping": [True]
        },
        "random_params": {
            "loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron"],
            "alpha": loguniform(0.0001, 0.1),
            "max_iter": randint(100, 10000),
            "early_stopping": [True]
        },
    }
}

knn_dict = {
    "knn_classifier": {
        "model": KNeighborsClassifier(),
        "grid_params": {
            "n_neighbors": list(range(1, 31)),
            "weights": ["uniform", "distance"],
            "algorithm": ["auto", "ball_tree", "kd_tree"],
            "leaf_size": list(range(10, 101, 5)),
            "metric": ["euclidean", "manhattan"],
        },
        "random_params": {
            "n_neighbors": list(range(1, 31)),
            "weights": ["uniform", "distance"],
            "algorithm": ["auto", "ball_tree", "kd_tree"],
            "leaf_size": list(range(10, 101, 5)),
            "metric": ["euclidean", "manhattan"],
        }
    }
}

scaled_models_list = (log_dict, svc_dict, sgdc_dict, knn_dict)
random_scaled_models = {k:v for model_dict in scaled_models_list for k,v in model_dict.items() if k != "grid_params"}
grid_scaled_models = {k:v for model_dict in scaled_models_list for k,v in model_dict.items() if k != "random_params"}

In [ ]:
randfor_dict = {
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "criterion": ["gini", "entropy", "log_loss"],
            "n_estimators": randint(10, 501),
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": randint(5, 51)
        }
    }
}

bayes_dict = {
    "bernoulli_bayes": {
        "model": BernoulliNB(),
        "params": {
            "alpha": uniform(0.1, 2.0),
            "fit_prior": [True, False],
            "binarize": uniform(0.0, 2.0)
        }
    }
}

grad_boost_dict = {
    "gradient_boosting": {
        "model": GradientBoostingClassifier(),
        "params": {
            "n_estimators": randint(100, 500),
            "learning_rate": loguniform(0.001, 0.1),
            "max_depth": randint(3, 10),
            "subsample": uniform(0.7, 1),
            "min_samples_split": randint(5, 20)
        }
    }
}


"""random_search_list = (log_dict, randfor_dict, bayes_dict, grad_boost_dict)
all_random_models = {k:v for model_dict in model_list for k,v in model_dict.items()}"""

### Defining search method arguments

f1 score is very common for clinical data as it indicates a balance between precision and recall

In [ ]:

random_search_args = {
    "search_model": RandomizedSearchCV,
    "search_params": {
        "cv":10, "n_iter":1000, "n_jobs":4, "scoring":{
            "f1": "f1",
            "AUC": "roc_auc",
            "Accuracy": "accuracy",
            "Recall": "recall"
        },
        "refit": "f1"
    }
}

grid_search_args = {
    "search_model": GridSearchCV,
    "search_params": {
        "cv":10, "scoring": "accuracy"
    }
}



### Training

In [ ]:
def cv_search(models_params, search_args, X_train, y_train):
    print("Given search args: ", search_args)
    scores_list = []
    search_engine = search_args["search_model"]
    search_params = search_args["search_params"]
    for model_name, mp in models_params.items():
        print(f"Starting cross-validation for {model_name}...")
        cv_classifier = search_engine(mp["model"], mp["random_params"], **search_params)
        start_time = time.time()    
        cv_classifier.fit(X_train, y_train)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Results for {model_name}:\n", cv_classifier.best_score_, cv_classifier.best_params_)
        scores_list.append({
            "model": model_name,
            "best_score": cv_classifier.best_score_,
            "best_params": cv_classifier.best_params_,
            "elapsed_time": elapsed_time,
            "cv_results": pd.DataFrame(cv_classifier.cv_results_)
        })
    return scores_list

def get_feature_weights(best_model, X, y, n_repeats, state, score):
    return permutation_importance(best_model, X, y, n_repeats=n_repeats, random_state=state, scoring=score)

def fit_test_model(X_train, X_test, y_train, y_test, features, best_model, best_model_params):
    best_model.set_params(**best_model_params)
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    print(f"Found best model: {type(best_model).__name__} with parameters: {best_model_params}")
    try:
        feature_weights = get_feature_weights(best_model, X_train, y_train, n_repeats=10, state=99, score="f1")
    except Exception:
        feature_weights = {}
    if feature_weights:
        importance_df = pd.DataFrame({y:x for x,y in sorted(zip(feature_weights["importances_mean"], features), reverse=True)}, index=[0])
        importance_df.to_csv(os.path.join(output_location, f"{type(best_model).__name__}_importance_df.csv"))
    else:
        print(f"Could not extract importance values for best model {type(best_model).__name__}:{best_model_params}")
    class_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
    class_report_df["roc_auc"] = roc_auc_score(y_test, y_pred_prob)
    print("Scores table: \n", class_report_df)
    class_report_df.to_csv(os.path.join(output_location, f"{type(best_model).__name__}_class_report.csv"))
    return class_report_df

def run_testing(X, y, models_dict, search_args, scaler=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)
    features = X_train.columns
    if scaler is not None:
        try:
            X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
            X_test = scaler.fit_transform(X_test)
            print(f"Scaled data with given scaler {scaler}")
        except Exception as e:
            print(f"Could not scale data with provided scaler {scaler}: {e}")
            return

    scores = cv_search(models_params=models_dict, search_args=search_args, X_train=X_train, y_train=y_train)

    print("Training finished")

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(scores, columns=["model", "best_score", "best_params", "elapsed_time"])
    results_df.to_csv(os.path.join(output_location, "results_df.csv"))
    for row in scores:
        model_name = row["model"]
        row["cv_results"].to_csv(os.path.join(output_location, f"{model_name}_cv_results.csv"))

    # Select the best model and evaluate it on the test set
    best_model_name = results_df.loc[results_df["best_score"].idxmax()]["model"]
    best_model_params = results_df.loc[results_df["best_score"].idxmax()]["best_params"]

    best_model = models_dict[best_model_name]["model"]
    class_report_df = fit_test_model(X_train, X_test, y_train, y_test, features, best_model, best_model_params)
    return results_df, class_report_df

search_args = random_search_args
output_location = "/home/pmata/mepram_dataframes/test8_results/"
models_dict = {"LogisticRegression": random_scaled_models["logistic_regression"]}

X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]

results_df, class_report_df = run_testing(
    X=X, y=y, models_dict=models_dict, search_args=search_args, scaler=MinMaxScaler()
)


# Plot scores distribution along cross-validation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_cv_results(cv_results_df, title):
    fig, ax = plt.subplots(figsize=(12,6))
    for col in ["mean_test_AUC", "mean_test_f1", "mean_test_Accuracy", "mean_test_Recall"]:
        sns.kdeplot(data=cv_results_df, x=col, label=col)
    ax.legend()
    plt.title(title)
    plt.tight_layout()
    plt.show()

for idx, row in results_df.iterrows():
    cv_results = pd.DataFrame(row["cv_results"])
    plot_cv_results(cv_results)


# Gradient Boost and other approaches

In [ ]:
from xgboost import XGBClassifier
from xgboost import cv as xgboost_cv
from xgboost import DMatrix
import optuna
import scipy.stats as stats

In [ ]:
xgb_dict = {
    "XGBoost_classifier": {
        "model": XGBClassifier(),
        "random_params": {
            "objective": ["binary:logistic"],
            'max_depth': stats.randint(3, 10),
            'learning_rate': stats.uniform(0.01, 0.1),
            'subsample': stats.uniform(0.5, 0.5),
            'n_estimators':stats.randint(50, 200)
        }
    }
}

In [ ]:
from sklearn.metrics import f1_score

search_args = random_search_args
output_location = "/home/pmata/mepram_dataframes/test_xgboost_results/"
param_grid = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 100],
    "learning_rate": [0.001, 0.1, 0.5],
    "n_estimators": [3,5, 10, 15],
    "max_depth": 5,
    "min_child_weight": 2,
    "gamma": 0.1,
    subsample=0.85,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27
} 
              
grid_search = GridSearchCV(estimator = XGBClassifier(),
    param_grid = param_grid,
    scoring='f1',
    n_jobs=4,
    cv=10,
    verbose=10
) 

X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

grid_search.fit(X_train, y_train)
print('Best Grid Search Parameters :',grid_search.best_params_)
print('Best Grid Search Score : ',grid_search.best_score_)


In [ ]:
import optuna
import xgboost as xgb
import joblib
from sklearn.metrics import f1_score, log_loss

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

def objective(trial):
    # Define the hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',  # Binary classification
        'eval_metric': 'auc',
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0, log=True),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 10)
    }
    
    # Train the model
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    # Predict and calculate accuracy
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred)
    return f1, logloss

# Create a study object and optimize the objective function
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=100)

# Print the best parameters and best score
print("Best parameters: ", study.best_params)
print("Best score: ", study.best_value)

joblib.dump(study, os.path.join(output_location, "xgboost_study.pkl"))

In [ ]:
import xgboost as xgb
processed_df["enf_hepaticas"] = processed_df["enf_hepaticas"].astype(int)
X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)
some_params = {'n_estimators': 252, 'max_depth': 13, 'learning_rate': 0.029417993022890093, 'subsample': 0.7878338148028812, 'colsample_bytree': 0.7735167084031885, 'gamma': 0.034664742223686935, 'min_child_weight': 2, 'reg_alpha': 0.11711315383329728, 'reg_lambda': 0.09837634948322023, 'scale_pos_weight': 2}
xxx = xgb.XGBClassifier(**some_params).fit(X_train, y_train, verbose=False)
y_pred = xxx.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import xgboost as xgb

study = joblib.load(os.path.join(output_location,"xgboost_study.pkl"))
print(study.best_params, study.best_value)

X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)
classif = xgb.XGBClassifier(**study.best_params).fit(X_train, y_train, verbose=False)
y_pred = classif.predict(X_test)
y_pred_prob = classif.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"Best model {type(classif)} with roc_auc = {roc_auc} and best params: {study.best_params}")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
import optuna.visualization as vis

vis.plot_optimization_history(study)
vis.plot_param_importances(study)
vis.plot_slice(study)